In [136]:
import numpy as np
import pandas as pd
import os
# import tensorflow as tf
import librosa
import note_seq
import bokeh
from note_seq import NoteSequence
from typing import Iterable, List, Tuple
from midi2audio import FluidSynth
import fluidsynth

import matplotlib.pyplot as plt

In [83]:
# np.random.uniform(1,128)

59.64386343211563

In [119]:
def gen_uni_notesequence(duration: int, time_step: int, max_num_pitches: int) -> List:
    ns = NoteSequence()
    num_parts = duration / time_step

    for i in np.arange(num_parts):

        for _ in np.arange(np.random.choice(max_num_pitches)):
            start_time = time_step * i
            end_time = start_time + np.random.uniform(0, time_step)
            end_time = end_time if end_time <= 100 else 100

            ns.notes.append(
                NoteSequence.Note(
                    pitch=np.random.choice(128),
                    velocity=np.random.choice(np.arange(1,128)),
                    start_time=start_time,
                    end_time=end_time,
                )
            )

    return ns


gen_midi = gen_uni_notesequence(duration=100, time_step=15, max_num_pitches=10)
note_seq.plot_sequence(gen_midi)

Loading BokehJS ...

In [125]:
# note_seq.note_sequence_to_midi_file(gen_midi, "./test_midi.mid")

features = ["midi_filename", "audio_filename"]
df = pd.DataFrame(columns=features)
df

,midi_filename,audio_filename


In [139]:
# midi_filename,audio_filename


def gen_synthetic_data(output_dir: str):

    os.makedirs(output_dir, exist_ok=True)

    path_gen_cvs = os.path.join(output_dir, "my_synthetic.csv")

    features = ["midi_filename", "audio_filename"]
    df = pd.DataFrame(columns=features)

    # fs = FluidSynth()

    midi_lst = []
    audio_lst = []
    for i in np.arange(3):
        gen_uni_ns = gen_uni_notesequence(duration=18, time_step=2, max_num_pitches=10)
        note_seq.note_sequence_to_midi_file(gen_uni_ns, f"{output_dir}/midi_{i}.mid")
        # fs.midi_to_audio(f"{output_dir}/midi_{i}.mid", f"{output_dir}/audio_{i}.wav")
        midi_lst.append(f"midi_{i}.mid")
        audio_lst.append(f"audio_{i}.wav")
    
    df["midi_filename"] = midi_lst
    df["audio_filename"] = audio_lst

    df.to_csv(os.path.join(output_dir, "my_synthetic.csv"), index=False)
    

    print(df)



output_dir = "/home/sibwa19/documents/main_course/second/project/MusicTranscription/data/synthetic/2022"

gen_synthetic_data(output_dir)

  midi_filename audio_filename
0    midi_0.mid    audio_0.wav
1    midi_1.mid    audio_1.wav
2    midi_2.mid    audio_2.wav
